### In this notebook we use LLama 3 70B and gpt4o for annotation

In [11]:
from rich import print
import json
from tqdm import tqdm
from dotenv import load_dotenv, find_dotenv
import os
from openai import OpenAI
import pandas as pd
from tqdm import tqdm

In [12]:
_ = load_dotenv(find_dotenv())
api_key = os.environ['OPENAI_API_KEY']

# base_url = os.environ['OPENAI_BASE_URL']
client = OpenAI()

In [13]:
pwd

'/notebooks/ABSA-for-Open-Ended-Qs-in-Education-Surveys/notebooks/label_cleansing'

In [14]:
example1 = "المحتوى أكثر من رائع"

In [15]:
system_message = """
سوف تتلقى مراجعة حول بعض الخدمات التعليمية، ومهمتك هي معرفة الموضوع الذي تتجذث عنه كل مراجعة.
قد تكون المراجعة باللغة الإنجليزية، أو باللغة العربية، أو كليهما.
خيارات الإخراج المسموح بها هي ما يلي:

* محتوى ومعلومات المقرر
* موازنة الجزء العملي مع الجزء النظري
* غير محدد
* سوق العمل
* المتطلبات و المهام و الدرجات
* دكتور المقرر
* الوقت و الجدول
* طريقة تدريس و تقديم المادة
* الجانب البحثي والأوراق البحثية
"""

In [16]:
user_template = """
عليك أن تستخرج الموضوع الخاص بالمراجعة التالية. 

## كيفية استخراج الموضع الخاص بالمراجعة:
* إذا كان نص المراجعة يتحدث عن المقرر الدراسي و المعلومات الواردة في هذا المقرر فإن الموضوع الخاص بهذه المراجعة هو: "محتوى ومعلومات المقرر"

* إذا كان نص المراجعة يتحدث عن الجزء العملي و أهميته و أهمية تناسبه مع الجزء النظري فإن الموضوع الخاص بهذه المراجعة هو: "موازنة الجزء العملي مع الجزء النظري"

* إذا كان نص المراجعة يتحدث عن سوق العمل و مدى ارتباط المنهج بسوق العمل فإن الموضوع الخاص يهذه المراجعة هو: "سوق العمل"

* إذا كان نص المراجعة يتحدث عن الأشياء المطلوب من الطلاب أن يقوموا بها أو عن درجات الطلاب فإن الموضوع الخاص بهذه المراجعة هو: "المتطلبات و المهام و الدرجات"

* إذا كان نص المراجعة يتحدث عن الدكتور الذي يشرح المادة فإن الموضوع الخاص بهذه المراجعة هو: "دكتور المقرر"

* إذا كان نص المراجعة يتحدث عن الخط الزمني لدراسة المادة أو عن جدول تدريس المادة و جدول الإمتحانات و التسليمات فإن الموضوع الخاص بهذه المراجعة هو: "الوقت و الجدول"

* إذا كان نص المراجعة يتحدث عن الطريقة التي يتم بها تقديم المادة للطلاب أو عن الطريقة التي يتم بها تدريس المادة للطلاب فإن الموضوع الخاص بهذه المراجعة هو: "طريقة تدريس و تقديم المادة"

* إذا كان نص المراجعة يتحدث عن الجانب البحثي للمادة أو عن الأوراق البحثية فإن الموضوع الخاص بهذه المراجعة هو: "الجانب البحثي و الأوراق البحثية"

* إذا لم يحتو نص المراجعة على أي من الموضوعات السابقة فإن موضوع هذه المراحعة هو: "غير محدد"

## ملحوظة: إذا وجدت أكثر من موضوع داخل المراجعة الواحدة يجب عليك استخراج الموضوع الموجود بكثرة داخل هذه المراجعة


"
المراجعة:
###
{}
###

## يجب عليك أن تكتب اسم الموضوع المستخرج فقط
output: 
"""

In [17]:
def format_user_message(template, text):
    return template.format(text)


In [18]:
def format_one_example(review):
    return {
        "messages": [
            {
                "role": "system",
                "content": system_message
            },
            {
                "role": "user",
                "content": format_user_message(user_template, review)
            }
        ]
    }

In [19]:
print(format_one_example(example1)['messages'][1]['content'])

عليك أن تستخرج الموضوع الخاص بالمراجعة التالية. 

## كيفية استخراج الموضع الخاص بالمراجعة:
* إذا كان نص المراجعة يتحدث عن المقرر الدراسي و المعلومات الواردة في هذا المقرر فإن الموضوع الخاص بهذه المراجعة هو:
"محتوى ومعلومات المقرر"

* إذا كان نص المراجعة يتحدث عن الجزء العملي و أهميته و أهمية تناسبه مع الجزء النظري فإن الموضوع الخاص بهذه المراجعة
هو: "موازنة الجزء العملي مع الجزء النظري"

* إذا كان نص المراجعة يتحدث عن سوق العمل و مدى ارتباط المنهج بسوق العمل فإن الموضوع الخاص يهذه المراجعة هو: "سوق 
العمل"

* إذا كان نص المراجعة يتحدث عن الأشياء المطلوب من الطلاب أن يقوموا بها أو عن درجات الطلاب فإن الموضوع الخاص بهذه 
المراجعة هو: "المتطلبات و المهام و الدرجات"

* إذا كان نص المراجعة يتحدث عن الدكتور الذي يشرح المادة فإن الموضوع الخاص بهذه المراجعة هو: "دكتور المقرر"

* إذا كان نص المراجعة يتحدث عن الخط الزمني لدراسة المادة أو عن جدول تدريس المادة و جدول الإمتحانات و التسليمات فإن 
الموضوع الخاص بهذه المراجعة هو: "الوقت و الجدول"

* إذا كان نص المراجعة يتحدث عن الطريقة التي يتم بها تقديم المادة للطلاب أو عن الطريقة التي يتم بها تدريس المادة 
للطلاب فإن الموضوع الخاص بهذه المراجعة هو: "طريقة تدريس و تقديم المادة"

* إذا كان نص المراجعة يتحدث عن الجانب البحثي للمادة أو عن الأوراق البحثية فإن الموضوع الخاص بهذه المراجعة هو: 
"الجانب البحثي و الأوراق البحثية"

* إذا لم يحتو نص المراجعة على أي من الموضوعات السابقة فإن موضوع هذه المراحعة هو: "غير محدد"

## ملحوظة: إذا وجدت أكثر من موضوع داخل المراجعة الواحدة يجب عليك استخراج الموضوع الموجود بكثرة داخل هذه المراجعة


"
المراجعة:
###
المحتوى أكثر من رائع
###

## يجب عليك أن تكتب اسم الموضوع المستخرج فقط
output:

In [20]:
# model_name = 'meta-llama/Meta-Llama-3-70B-Instruct'
# model_name = 'meta-llama/Meta-Llama-3-70B-Instruct'
model_name = 'gpt-4o'
def get_completion(review, model_name):
    completion = client.chat.completions.create(
        model=model_name,
        temperature=0,
        messages=format_one_example(review)['messages']
    )
    
    return completion.choices[0].message.content

In [21]:
def predict_gpt(item, model_name=model_name):
    label = get_completion(item, model_name)
    # result = json.loads(completion)
    # pred_labels = [result.get(token, 'O') for token in item['tokens']]
    # pred_labels = [v for k, v in result.items()]
    # organizations = result['Organizations']
    return label

In [22]:
example1 = "المحتوى أكثر من رائع"

In [23]:
predict_gpt(example1)

'محتوى ومعلومات المقرر'

In [24]:
train_data_path = '../../data/result/train_df.csv'
test_data_path = '../../data/result/test_df.csv'
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

In [25]:
len(['موازنة الجزء العملي مع الجزء النظري' ,'محتوى ومعلومات المقرر',
 'الوقت و الجدول' ,'دكتور المقرر' ,'طريقة تدريس و تقديم المادة', 'غير محدد',
 'سوق العمل' ,'المتطلبات و المهام و الدرجات',
 'الجانب البحثي و الأوراق البحثية'])



9

In [26]:
train_data.head()

,column,text,topic_id,topic_name
0,like,أكثر ما أعجبني: التطبيق العملي المحدث والمشابه...,3,موازنة الجزء العملي مع الجزء النظري
1,improve_course,اقتراحاتي للتحسين: التطبيق والبعد عن التدريس ا...,3,موازنة الجزء العملي مع الجزء النظري
2,like,أكثر ما أعجبني: the content,0,محتوى ومعلومات المقرر
3,improve_course,اقتراحاتي للتحسين: provide more new books,3,موازنة الجزء العملي مع الجزء النظري
4,improve_course,اقتراحاتي للتحسين: زيادة ساعات العملي,8,الوقت و الجدول


In [31]:
tqdm.pandas()

print('Annotate train data...')
train_data['topic_llm'] = train_data['text'].progress_apply(predict_gpt)
print('Annotate test data...')
test_data['topic_llm'] = test_data['text'].progress_apply(predict_gpt)

Annotate train data...

100%|██████████| 951/951 [09:39<00:00,  1.64it/s] 


Annotate test data...

100%|██████████| 634/634 [05:45<00:00,  1.83it/s]


In [30]:
# for e in train_data['text'].sample(50):
#     # print(e)
#     print('topic:\n', predict_gpt(e))
#     print('----'*25)
    

In [18]:
# train_data['sentiment'] = train_data['sentiment'].progress_apply(str.capitalize)
# test_data['sentiment'] = test_data['sentiment'].progress_apply(str.capitalize)

In [32]:
train_data.head()

,column,text,topic_id,topic_name,topic_llm
0,like,أكثر ما أعجبني: التطبيق العملي المحدث والمشابه...,3,موازنة الجزء العملي مع الجزء النظري,موازنة الجزء العملي مع الجزء النظري
1,improve_course,اقتراحاتي للتحسين: التطبيق والبعد عن التدريس ا...,3,موازنة الجزء العملي مع الجزء النظري,طريقة تدريس و تقديم المادة
2,like,أكثر ما أعجبني: the content,0,محتوى ومعلومات المقرر,محتوى ومعلومات المقرر
3,improve_course,اقتراحاتي للتحسين: provide more new books,3,موازنة الجزء العملي مع الجزء النظري,محتوى ومعلومات المقرر
4,improve_course,اقتراحاتي للتحسين: زيادة ساعات العملي,8,الوقت و الجدول,موازنة الجزء العملي مع الجزء النظري


In [34]:
len(train_data['topic_llm'].value_counts().index)

9

In [35]:
train_data['topic_llm'] = train_data['topic_llm'].apply(lambda x: "الجانب البحثي و الأوراق البحثية" if x == 'الجانب البحثي والأوراق البحثية' else x)
test_data['topic_llm'] = test_data['topic_llm'].apply(lambda x: "الجانب البحثي و الأوراق البحثية" if x == 'الجانب البحثي والأوراق البحثية' else x)

In [36]:
train_data['topic_llm'].value_counts() / len(train_data)

محتوى ومعلومات المقرر                  0.228181
دكتور المقرر                           0.187171
موازنة الجزء العملي مع الجزء النظري    0.180862
غير محدد                               0.149317
المتطلبات و المهام و الدرجات           0.084122
طريقة تدريس و تقديم المادة             0.056782
الوقت و الجدول                         0.043113
الجانب البحثي و الأوراق البحثية        0.036803
سوق العمل                              0.033649
Name: topic_llm, dtype: float64

In [39]:
test_data['topic_llm'].value_counts() # / len(test_data)

محتوى ومعلومات المقرر                  140
دكتور المقرر                           125
موازنة الجزء العملي مع الجزء النظري    105
غير محدد                               100
المتطلبات و المهام و الدرجات            53
طريقة تدريس و تقديم المادة              36
الوقت و الجدول                          26
الجانب البحثي و الأوراق البحثية         25
سوق العمل                               24
Name: topic_llm, dtype: int64

In [38]:
train_save_path = '../../data/result/train_with_topic_df_llm_gpt4o.csv'
test_save_path = '../../data/result/test_with_topic_df_llm_gpt4o.csv'

if os.path.exists(train_save_path):
    print('The path exists!')
else:
    print('Saving...')
    train_data.to_csv(train_save_path, index=False)
    

if os.path.exists(test_save_path):
    print('The path exists!')
else:
    print('Saving...')
    test_data.to_csv(test_save_path, index=False)

Saving...

Saving...